In [ ]:
## Load necessary modules

import asf_search as asf
import geopandas as gpd
import pandas as pd

import numpy as np
from netrc import netrc
from subprocess import Popen
from platform import system
from getpass import getpass
import folium
import datetime as dt
from shapely.geometry import box
from shapely.geometry import Point
import shapely.wkt as wkt
import rioxarray
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cartopy.crs as ccrs
import xarray as xr
from rasterio.plot import show


# %watermark --iversions

import os

In [ ]:
## Avoid lots of these warnings printing to notebook from asf_search
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Generates authentication token
# Asks for your Earthdata username and password for first time, if netrc does not exists.

urs = 'urs.earthdata.nasa.gov'    # Earthdata URL endpoint for authentication
prompts = ['Enter NASA Earthdata Login Username: ',
           'Enter NASA Earthdata Login Password: ']

# Determine the OS (Windows machines usually use an '_netrc' file)
netrc_name = "_netrc" if system()=="Windows" else ".netrc"

# Determine if netrc file exists, and if so, if it includes NASA Earthdata Login Credentials
try:
    netrcDir = os.path.expanduser(f"~/{netrc_name}")
    netrc(netrcDir).authenticators(urs)[0]

# Below, create a netrc file and prompt user for NASA Earthdata Login Username and Password
except FileNotFoundError:
    homeDir = os.path.expanduser("~")
    Popen('touch {0}{2} | echo machine {1} >> {0}{2}'.format(homeDir + os.sep, urs, netrc_name), shell=True)
    Popen('echo login {} >> {}{}'.format(getpass(prompt=prompts[0]), homeDir + os.sep, netrc_name), shell=True)
    Popen('echo \'password {} \'>> {}{}'.format(getpass(prompt=prompts[1]), homeDir + os.sep, netrc_name), shell=True)
    # Set restrictive permissions
    Popen('chmod 0600 {0}{1}'.format(homeDir + os.sep, netrc_name), shell=True)

    # Determine OS and edit netrc file if it exists but is not set up for NASA Earthdata Login
except TypeError:
    homeDir = os.path.expanduser("~")
    Popen('echo machine {1} >> {0}{2}'.format(homeDir + os.sep, urs, netrc_name), shell=True)
    Popen('echo login {} >> {}{}'.format(getpass(prompt=prompts[0]), homeDir + os.sep, netrc_name), shell=True)
    Popen('echo \'password {} \'>> {}{}'.format(getpass(prompt=prompts[1]), homeDir + os.sep, netrc_name), shell=True)


In [ ]:
## Enter user-defined parameters
aoi = Point(-22.3709, 63.9031) #box(-155.88, 19.0, -155.014, 20.1962)     # (W, S, E, N)
wavelength = 0.056
orbitPass = "DESCENDING"
pathNumber = 155
referenceDate = dt.datetime.fromisoformat('2024-02-01 00:00:00')         #'YYYY-MM-DD HH:MM:SS'
secondaryDate = dt.datetime.fromisoformat('2024-10-30 23:59:59')         #'YYYY-MM-DD HH:MM:SS'
savedir = './'

In [ ]:
## Search for OPERA RTC data in ASF DAAC
search_params = dict(
    intersectsWith= aoi.wkt,
    dataset='OPERA-S1',
    processingLevel='RTC',
    # flightDirection = orbitPass,
    start=referenceDate,
    end=secondaryDate
)

## Return results
results = asf.search(**search_params)
len(results)

In [ ]:
## Display the queried data
gf = gpd.GeoDataFrame.from_features(results.geojson(), crs='EPSG:4326')
gf.head()

In [ ]:
gf.keys()

In [ ]:
## Filter data based on specified track number
# gf = gf[gf.pathNumber==pathNumber]
rtc_df = gf[['operaBurstID', 'fileID', 'polarization', 'startTime', 'stopTime', 'url', 'additionalUrls', 'geometry']]
rtc_df['startTime'] = pd.to_datetime(rtc_df.startTime).dt.date
rtc_df['stopTime'] = pd.to_datetime(rtc_df.stopTime).dt.date
rtc_df = rtc_df.drop_duplicates(subset=['operaBurstID', 'startTime'], ignore_index=True)
rtc_df

In [ ]:
rtc_df = rtc_df.sort_values(by=["startTime"], ignore_index=True)
rtc_df

In [ ]:
## Basemap function for Folium
def getbasemaps():
    # Add custom base maps to folium
    basemaps = {
        'Google Satellite Hybrid': folium.TileLayer(
            tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
            attr = 'Google',
            name = 'Google Satellite',
            overlay = True,
            control = True,
            # opacity = 0.8,
            show = False
        ),
        'Esri Satellite': folium.TileLayer(
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            overlay = True,
            control = True,
            #opacity = 0.8,
            show = False
        )
    }
    return basemaps

In [ ]:
# Interactive map to visualize the boundaries of the selected OPERA CSLCs
m = folium.Map(location=[aoi.centroid.y, aoi.centroid.x], zoom_start=8, tiles="CartoDB positron")

# Add custom basemaps
basemaps = getbasemaps()
for basemap in basemaps:
    basemaps[basemap].add_to(m)

# layer Control
m.add_child(folium.LayerControl())

## RLE sites
for _, r in rtc_df.iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"color": "red"}, control=False)
    folium.Popup(r["operaBurstID"]).add_to(geo_j)
    geo_j.add_to(m)

m

In [ ]:
## Download the relevant OPERA RTCs to a temporary "rtc" folder 
os.makedirs(f'{savedir}/rtc', exist_ok='True')
asf.download_urls(urls=rtc_df.url, path=f'{savedir}/rtc', processes=10)
asf.download_urls(urls=rtc_df.additionalUrls.explode(ignore_index=True), path=f'{savedir}/rtc', processes=10)

In [ ]:
def read_rtc(tiffile):
    rtc = xr.open_dataset(tiffile)
    epsg = rtc.rio.crs.to_proj4()

    return rtc, epsg

In [ ]:
aoi = 'POLYGON((-22.5124 63.8204,-22.3171 63.8204,-22.3171 63.9565,-22.5124 63.9565,-22.5124 63.8204))' #'POLYGON((-22.4965 63.8422,-22.3054 63.8422,-22.3054 63.9227,-22.4965 63.9227,-22.4965 63.8422))'
aoi_xorigin = wkt.loads(aoi).bounds[0]
aoi_yorigin = wkt.loads(aoi).bounds[1]
aoi_dx = np.abs(wkt.loads(aoi).bounds[2] - wkt.loads(aoi).bounds[0])
aoi_dy = np.abs(wkt.loads(aoi).bounds[3] - wkt.loads(aoi).bounds[1])

In [ ]:
#load rtc
vv_stack = []; vv_dates = []; vh_stack = []; ratio_stack = []
for i, fileID, start_date, bounding_polygon in zip(rtc_df.index, rtc_df.fileID, rtc_df.startTime, rtc_df.geometry):
    # Get VV
    vv, epsg = read_rtc(f"{savedir}/rtc/{fileID}_VV.tif")
    vv = vv.rio.clip([wkt.loads(aoi)], "epsg:4326")
    vv_ = 10*np.log10(vv.band_data.values[0])
    vv_stack.append(vv_)
    vv_dates.append(pd.to_datetime(start_date).date())

    # Plot
    rtc_poly = bounding_polygon
    bbox = [rtc_poly.bounds[0], rtc_poly.bounds[2], rtc_poly.bounds[1], rtc_poly.bounds[3]]
    
    fig, ax = plt.subplots(figsize=(12, 3))
    cax=ax.imshow(vv_, cmap='gray',interpolation=None, origin='upper', extent=bbox, vmin=np.nanpercentile(vv_,2), vmax=np.nanpercentile(vv_,98))
    # roi = patches.Rectangle((aoi_xorigin,aoi_yorigin),aoi_dx,aoi_dy,edgecolor='red', facecolor='none')
    # ax.add_patch(roi)
    cbar = fig.colorbar(cax,orientation='vertical',fraction=0.01,pad=0.02)
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title(f'VV_{pd.to_datetime(start_date).date()}',fontsize=12)

    ## Get VH
    vh, epsg = read_rtc(f"{savedir}/rtc/{fileID}_VH.tif")
    vh = vh.rio.clip([wkt.loads(aoi)], "epsg:4326")
    vh_ = 10*np.log10(vh.band_data.values[0])
    vh_stack.append(vh_)

    # Plot    
    fig, ax = plt.subplots(figsize=(12, 3))
    cax=ax.imshow(vh_, cmap='gray',interpolation=None, origin='upper', extent=bbox, vmin=np.nanpercentile(vh_,2), vmax=np.nanpercentile(vh_,98))
    # roi = patches.Rectangle((aoi_xorigin,aoi_yorigin),aoi_dx,aoi_dy,edgecolor='red', facecolor='none')
    # ax.add_patch(roi)
    cbar = fig.colorbar(cax,orientation='vertical',fraction=0.01,pad=0.02)
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title(f'VH_{pd.to_datetime(start_date).date()}',fontsize=12)

    # Calculate for ratio
    ratio = np.log10(vv_/vh_)
    ratio_stack.append(ratio)

    # Plot    
    fig, ax = plt.subplots(figsize=(12, 3))
    cax=ax.imshow(ratio, cmap='gray',interpolation=None, origin='upper', extent=bbox, vmin=np.nanpercentile(ratio,2), vmax=np.nanpercentile(ratio,98))
    # roi = patches.Rectangle((aoi_xorigin,aoi_yorigin),aoi_dx,aoi_dy,edgecolor='red', facecolor='none')
    # ax.add_patch(roi)
    cbar = fig.colorbar(cax,orientation='vertical',fraction=0.01,pad=0.02)
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title(f'RATIO_{pd.to_datetime(start_date).date()}',fontsize=12)

In [ ]:
vv_band = xr.DataArray(vv_stack, coords=dict(time=vv_dates, y=vv.y, x=vv.x, spatial_ref=vv.spatial_ref), dims=("time", "y", "x"))
vh_band = xr.DataArray(vh_stack, coords=dict(time=vv_dates, y=vv.y, x=vv.x, spatial_ref=vv.spatial_ref), dims=("time", "y", "x"))
ratio_band = xr.DataArray(ratio_stack, coords=dict(time=vv_dates, y=vv.y, x=vv.x, spatial_ref=vv.spatial_ref), dims=("time", "y", "x"))

band_var = xr.Variable('band', range(1,4))
combined = xr.concat([vv_band, vh_band, ratio_band], dim=band_var)
combined

# Create Geocube

In [ ]:
ds = combined.copy()
ds.rio.write_crs(f"epsg:{epsg.split(':')[-1]}", inplace=True)

In [ ]:
from pyproj import Transformer
xv, yv = np.meshgrid(ds.x, ds.y)

transformer = Transformer.from_crs(f"epsg:{epsg.split(':')[-1]}", 
                                   "epsg:4326",
                                   always_xy=True,
                                  ) 

lon, lat = transformer.transform(xv, yv)
ds.coords['lon'] = (('y', 'x'), lon)
ds.coords['lat'] = (('y', 'x'), lat)
ds.attrs['crs']  = '+init=epsg:4326'

In [ ]:
rgb_bands = [1,2,3]
ds_crop = ds.rio.clip([wkt.loads(aoi)], "epsg:4326")
ds_crop.sel(band=rgb_bands).plot.imshow(col='time',add_colorbar=True, robust=True, col_wrap=5)

In [ ]:
ds.sel(band=1).rio.clip([wkt.loads(aoi)], "epsg:4326").plot(col='time', robust=True, y='lat', x='lon', col_wrap=5)

In [ ]:
ds.sel(band=2).rio.clip([wkt.loads(aoi)], "epsg:4326").plot(col='time', robust=True, y='lat', x='lon', col_wrap=5)

In [ ]:
ds.sel(band=3).rio.clip([wkt.loads(aoi)], "epsg:4326").plot(col='time', y='lat', x='lon', robust=True, col_wrap=5)

In [ ]:
date12_vv = ds.sel(band=1).rio.clip([wkt.loads(aoi)], "epsg:4326").isel(time=1) - ds.sel(band=1).rio.clip([wkt.loads(aoi)], "epsg:4326").isel(time=-4)
date12_vh = ds.sel(band=2).rio.clip([wkt.loads(aoi)], "epsg:4326").isel(time=1)  - ds.sel(band=2).rio.clip([wkt.loads(aoi)], "epsg:4326").isel(time=-4)
date12_ratio = ds.sel(band=3).rio.clip([wkt.loads(aoi)], "epsg:4326").isel(time=1)  - ds.sel(band=3).rio.clip([wkt.loads(aoi)], "epsg:4326").isel(time=-4)

In [ ]:
date12_vv.plot(y='lat', x='lon', levels=10, cmap='RdBu_r', robust=True, figsize=(8,6)); plt.title(f"Log Difference: VV ({vv_dates[1].strftime('%Y%m%d')}-{vv_dates[-4].strftime('%Y%m%d')})")

In [ ]:
date12_vh.plot(y='lat', x='lon', levels=10, cmap='RdBu_r', robust=True, figsize=(8,6)); plt.title(f"Log Difference: VH ({vv_dates[1].strftime('%Y%m%d')}-{vv_dates[-4].strftime('%Y%m%d')})")

In [ ]:
date12_ratio.plot(y='lat', x='lon', levels=10, cmap='RdBu_r', robust=True, figsize=(8,6)); plt.title(f"Log Difference: VV/VH ({vv_dates[1].strftime('%Y%m%d')}-{vv_dates[-4].strftime('%Y%m%d')})")

In [ ]:
before = (ds.sel(band=1).rio.clip([wkt.loads(aoi)], "epsg:4326").isel(time=slice(0, len(ds.time)-1)))
after = (ds.sel(band=1).rio.clip([wkt.loads(aoi)], "epsg:4326").isel(time=slice(1, len(ds.time))))
# diff = before.values-after.values        ## sequential analysis
diff = np.tile(before.values[0],(len(after), 1,1))-after.values        ##single reference

In [ ]:
time = []
for i in range(0,len(ds.time)-1):
    # time.append(f"{ds.time.values[i].strftime('%Y%m%d')}-{ds.time.values[i+1].strftime('%Y%m%d')}")   ## for sequential analysis
    time.append(f"{ds.time.values[0].strftime('%Y%m%d')}-{ds.time.values[i+1].strftime('%Y%m%d')}")     ## single reference date

time

In [ ]:
diff_vv = xr.DataArray(diff, coords=dict(date=time, y=before.lat, x=before.lon, spatial_ref=before.spatial_ref), dims=("date", "y", "x"))
diff_vv.plot(col='date', robust=True, y='y', x='x', col_wrap=4)

In [ ]:
before = (ds.sel(band=2).rio.clip([wkt.loads(aoi)], "epsg:4326").isel(time=slice(0, len(ds.time)-1)))
after = (ds.sel(band=2).rio.clip([wkt.loads(aoi)], "epsg:4326").isel(time=slice(1, len(ds.time))))
# diff = before.values-after.values        ## sequential analysis
diff = np.tile(before.values[0],(len(after), 1,1))-after.values        ##single reference

In [ ]:
diff[np.where(diff>-0.7)] = np.nan
diff_vh = xr.DataArray(diff, coords=dict(date=time, y=before.lat, x=before.lon, spatial_ref=before.spatial_ref), dims=("date", "y", "x"))
diff_vh.plot(col='date', robust=True, y='y', x='x', col_wrap=4, cmap='RdBu')

In [ ]:
before = (ds.sel(band=3).rio.clip([wkt.loads(aoi)], "epsg:4326").isel(time=slice(0, len(ds.time)-1)))
after = (ds.sel(band=3).rio.clip([wkt.loads(aoi)], "epsg:4326").isel(time=slice(1, len(ds.time))))
# diff = before.values-after.values        ## sequential analysis
diff = np.tile(before.values[0],(len(after), 1,1))-after.values        ##single reference

In [ ]:
diff_ratio = xr.DataArray(diff, coords=dict(date=time, y=before.lat, x=before.lon, spatial_ref=before.spatial_ref), dims=("date", "y", "x"))
diff_ratio.plot(col='date', robust=True, y='y', x='x', col_wrap=4)